# 2019-2021 CAO data analysis
***

1. Save original data file
2. Load CAO data (2019-2021) into spreadsheet
    - 2021: from website to csv file
    - 2020: from excel to csv file
    - 2019: from pdf to csv file
3. Clean data, ensuring data integrity 
4. Analysis
    - plots of overview by year
    - overall points (sum/average) + plots
    - courses that has raised points by year
    - 


In [1]:
# using Anaconda to install Camelot
#$ conda install -c conda-forge camelot-py

In [2]:
# request data from website for 2021 data
import requests as rq

# Data was in a particular pattern, extract using regular expression
import re 

# date time for file naming
import datetime as dt

# data frame
import pandas as pd

# downloading 2020 data
import urllib.request as urlrq

# convert pdf to csv/df for 2019 data
import camelot as cam 

# for filename matching for 2019 data
import glob

In [3]:
# get current date and time, for file path creation later
now = dt.datetime.now()

#format as a string
nowstr = now.strftime("%Y%m%d_%H%M%S")

## 2021 CAO data 
link to CAO 2021 data website : http://www2.cao.ie/points/l8.php <br/>
***
*Steps to turn data from online source to data frame:*
1. Get data from online source, then save as html with correct encoding
2. Extract relevant data using Regular Expression
3. Save extracted data as csv file then read in as data frame

Using Regular Expression to search and extract data, converting raw HTML to csv

In [4]:
# Get data from CAO website to notebook
points = rq.get("http://www2.cao.ie/points/l8.php")
points
# <Response [200]> means successfully getting the data

<Response [200]>

**Saving original data set** <br>
Using datatime string created earlier to name and identify files saved at different point of time

In [5]:
# create file path with date time to save the data 
pathHTML = "caoData2021_"+nowstr+".html"

In [6]:
# loop through the lines to have a look of the content
for line in points.iter_lines():
    print (line)

b'<html>'
b'<BODY>'
b'<a name="deg"><hr>'
b'<h3><center> Points Required for Entry to 2021 Level 8 Courses</h3></center><p>'
b'<A HREF="l76.php">Go to Level 7/6 List of Institutions.</a><p>'
b'<PRE><b>'
b'*     Not all on this points score were offered places.'
b'#     Test / Interview / Portfolio / Audition'
b'AQA   All qualified applicants'
b'v     New competition for available places'
b' '
b'</PRE>'
b'<dl>'
b'<p>'
b'<dt><a href="#ac8"><b>American College Dublin</b></a>'
b'<dt><a href="#al8"><b>Athlone Institute of Technology</b></a>'
b'<dt><a href="#by8"><b>IBAT College Dublin</b></a>'
b'<dt><a href="#cm8"><b>Marino Institute of Education</b></a>'
b'<dt><a href="#pc8"><b>Carlow College</b></a>'
b'<dt><a href="#ct8"><b>CCT College Dublin</b></a>'
b'<dt><a href="#cr8"><b>Cork Institute of Technology</b></a>'
b'<dt><a href="#db8"><b>Dublin Business School</b></a>'
b'<dt><a href="#dc8"><b>Dublin City University</b></a>'
b'<dt><a href="#dk8"><b>Dundalk Institute of Technology</b></a>'
b'

Before digging into extracting relevant data, we have to fix the decoding error first. <br/>
After some explorations, there is an error in decoding text. The server uses iso-8859-1 however it does not decode \x96, therefore we use standard cp1252 which is very similar but will decode \x96.

In [7]:
# change encoding to "cp1252"
original_encoding = points.encoding
points.encoding = 'cp1252'

In [8]:
# save the original html file
with open(pathHTML, "w") as f:
    f.write(points.text)

**Extract relevant data** <br>
We will need to get rid of the html tags and extract only the course code, course name, points for round 1 and 2.

There seems to be a pattern to the data that we need, Using Regular Expression (Regex) to match the pattern.

In [9]:
# pattern of the data we want to extract
# course code consisting of 2 uppercase letters and 3 integers
regex = r'([A-Z]{2}[0-9]{3})(.*)'
# compile the regex to match the lines
re_course = re.compile(regex)

We will now move on to iterating all the lines in the html, with the correct encoding and extracting relevant data using regular expression. The extracted data will then be "split" and "join" into csv ready format and saved in a csv file. 

In [10]:
# Save the file into csv using the path pattern
path2021 = "caoData2021_csv_" + nowstr + ".csv"

# Keep track of courses processed
no_lines = 0

# Open the csv file for writing
with open (path2021,"w") as f:
    # write head row, string join method, with commas separting the columns for csv
    f.write(','.join(['code','title','pointsR1','pointsR2'])+'\n')
    #loop through the lines
    for line in points.iter_lines():
        #decode the line using wrong encoding
        dline = line.decode("cp1252")
        #match lines that represent courses 
        if re_course.fullmatch(dline):
            #add 1 to line counter
            no_lines = no_lines + 1
            # The course code.
            course_code = dline[:5]
            # The course title.
            course_title = dline[7:57].strip()
            # Round one points.
            course_points = re.split(' +', dline[60:])
            if len(course_points) != 2:
                course_points = course_points[:2]
            # Join the fields using a comma.
            linesplit = [course_code, course_title, course_points[0], course_points[1]]
            # Rejoin the substrings with commas in between.
            f.write(','.join(linesplit) + '\n')

# Print total number of processes lines
print(f"Total number of lines is {no_lines}.")

Total number of lines is 949.


In [11]:
df2021 = pd.read_csv(path2021, encoding="cp1252")

In [12]:
df2021

# there's a lot of "NaN" in pointsR2, 
# because not all courses went through R2
# might worth looking into number and courses that went through R2

,code,title,pointsR1,pointsR2
0,AL801,Software Design for Virtual Reality and Gaming,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructure,321,NaN
4,AL810,Quantity Surveying,328,NaN
...,...,...,...,...
944,WD211,Creative Computing,270,NaN
945,WD212,Recreation and Sport Management,262,NaN
946,WD230,Mechanical and Manufacturing Engineering,230,230
947,WD231,Early Childhood Care and Education,266,NaN


## 2020 data - save original and load into csv file

In [13]:
url2020 = "http://www2.cao.ie/points/CAOPointsCharts2020.xlsx"

#### Save original File

In [14]:
# create file path
pathxlsx = 'cao2020_' + nowstr + '.xlsx'

In [15]:
urlrq.urlretrieve(url2020, pathxlsx)

('cao2020_20211229_053530.xlsx', <http.client.HTTPMessage at 0x1d19ac99550>)

#### Load into pandas

In [16]:
# first 10 rows are preemble, don't need them in the data frame
df2020 = pd.read_excel(url2020, skiprows=10)
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# spot check
df2020.iloc[-1]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

In [18]:
# create a file path for pandas data
path2020 = "cao2020_" + nowstr + ".csv"

In [19]:
# save it.
df2020.to_csv(path2020)

## 2019 data - save original and load into csv file

reference: <br>
https://www.analyticsvidhya.com/blog/2020/08/how-to-extract-tabular-data-from-pdf-document-using-camelot-in-python/
https://camelot-py.readthedocs.io/en/master/user/quickstart.html


**Steps to turn data from pdf file to data frame**
1. 

In [44]:
# read in 2019 data from pdf using camelot
table2019 = cam.read_pdf("CAO data/caoData2019_original.pdf", pages='all', flavor='lattice', split_text=True)

In [45]:
# To see how many tables in the pdf
table2019
# camelot separted the table into 18 tables, as there were 18 pages


<TableList n=18>

In [22]:
# parsing report shows the accuracy of extraction
for i in range(18):
    print(table2019[i].parsing_report)

# data were most likely extracted correctly

{'accuracy': 99.58, 'whitespace': 6.91, 'order': 1, 'page': 1}
{'accuracy': 100.0, 'whitespace': 4.02, 'order': 1, 'page': 2}
{'accuracy': 100.0, 'whitespace': 3.57, 'order': 1, 'page': 3}
{'accuracy': 100.0, 'whitespace': 6.7, 'order': 1, 'page': 4}
{'accuracy': 100.0, 'whitespace': 3.12, 'order': 1, 'page': 5}
{'accuracy': 100.0, 'whitespace': 2.68, 'order': 1, 'page': 6}
{'accuracy': 100.0, 'whitespace': 4.46, 'order': 1, 'page': 7}
{'accuracy': 100.0, 'whitespace': 7.59, 'order': 1, 'page': 8}
{'accuracy': 97.37, 'whitespace': 10.71, 'order': 1, 'page': 9}
{'accuracy': 100.0, 'whitespace': 3.57, 'order': 1, 'page': 10}
{'accuracy': 100.0, 'whitespace': 4.02, 'order': 1, 'page': 11}
{'accuracy': 100.0, 'whitespace': 2.68, 'order': 1, 'page': 12}
{'accuracy': 100.0, 'whitespace': 2.68, 'order': 1, 'page': 13}
{'accuracy': 100.0, 'whitespace': 5.36, 'order': 1, 'page': 14}
{'accuracy': 100.0, 'whitespace': 2.68, 'order': 1, 'page': 15}
{'accuracy': 99.92, 'whitespace': 6.7, 'order': 1

In [46]:
# each table is a pandas dataframe
# first page has 47 rows and 4 columns
table2019[0]
table2019[0].df

,0,1,2,3
0,Course Code,INSTITUTION and COURSE,EOS,Mid
1,,Athlone Institute of Technology,,
2,AL801,Software Design with Virtual Reality and Gaming,304,328
3,AL802,Software Design with Cloud Computing,301,306
4,AL803,Software Design with Mobile Apps and Connected...,309,337
5,AL805,Network Management and Cloud Infrastructure,329,442
6,AL810,Quantity Surveying,307,349
7,AL820,Mechanical and Polymer Engineering,300,358
8,AL830,General Nursing,410,429
9,AL832,Psychiatric Nursing,387,403


In [47]:
# concat all tables into 1 df
for table in table2019:
    print(table.df)

              0                                                  1     2    3
0   Course Code                             INSTITUTION and COURSE   EOS  Mid
1                                  Athlone Institute of Technology           
2         AL801    Software Design with Virtual Reality and Gaming   304  328
3         AL802               Software Design with Cloud Computing   301  306
4         AL803  Software Design with Mobile Apps and Connected...   309  337
5         AL805        Network Management and Cloud Infrastructure   329  442
6         AL810                                 Quantity Surveying   307  349
7         AL820                 Mechanical and Polymer Engineering   300  358
8         AL830                                    General Nursing   410  429
9         AL832                                Psychiatric Nursing   387  403
10        AL836                       Nutrition and Health Science   352  383
11        AL837            Sports Science with Exercise Physiolo

The following code cell for exporting the data frame to csv files. The 18-page pdf file will be exported as a zip file with 18 csv files. The `compress` parameter compress them into 1 zip file. The data frame can also be exported as excel, html, json etc. 

In [34]:
# export the tables to 18 csv files inside a zip file (compressed) 
#table2019.export(CAO2019.csv", f="csv", compress=True)

### Concat and join data of 2020 and 2021
https://pandas.pydata.org/docs/getting_started/intro_tutorials/08_combine_dataframes.html
join them by course code and course title, drop the duplicated rows, then join the points by code and title

In [ ]:
courses2021 = df2021[["code", "title"]]
courses2021

In [ ]:
courses2020 = df2020[["COURSE CODE2","COURSE TITLE"]]
#change the name of columns to match the one in 2021
courses2020.columns=["code","title"]
courses2020

In [ ]:
allcourses = pd.concat([courses2021, courses2020], ignore_index=True)
allcourses

In [ ]:
allcourses.iloc[175]

In [ ]:
allcourses.iloc[949]

<br>
After concatenating, there will be duplicated rows from data of 2 different years. In order to remove duplicates, match the courses by course code as course code should be unique to a particular course in a particular HEI.
<br>

In [ ]:
allcourses.sort_values("code")

In [ ]:
# Finds all extra copies of duplicated rows.
allcourses[allcourses.duplicated()]

In [ ]:
# Returns a copy of the data frame with only distinct rows.
allcourses.drop_duplicates()

In [ ]:
# these are the extra copies of duplicated rows
allcourses[allcourses.duplicated(subset=["code"])]

In [ ]:
# Returns a data frame with only distinct course code.
allcourses.drop_duplicates(subset=['code'], inplace=True, ignore_index=True)

In [ ]:
allcourses

In [ ]:
# set course code as index (2021 data)
df2021.set_index('code', inplace=True)
df2021.columns = ['title', 'points_r1_2021', 'points_r2_2021']
df2021

In [ ]:
# Set course code as index(2021 & 2020).
allcourses.set_index('code', inplace=True)

In [ ]:
allcourses = allcourses.join(df2021[['points_r1_2021']])
allcourses

In [ ]:
# change the names of column in df2020
df2020_r1 = df2020[['COURSE CODE2', 'R1 POINTS']]
df2020_r1.columns = ['code', 'points_r1_2020']
df2020_r1

In [ ]:
# set course code as index (2020).
df2020_r1.set_index('code', inplace=True)
df2020_r1

In [ ]:
# Join 2020 points to allcourses.
allcourses = allcourses.join(df2020_r1)
allcourses

### Concat and join data of 2019 and 2020



## Analysis
***

Describe and summary plots of 3 years of data
- max, min, mean, std points
- course with highest and lowest points
- 

In [ ]:
allcourses.describe(include="all") 

Comparing data of 2019,2020, 2021 using pandas functions

Create plots and visualizations

References:
[1]
[2]